In [45]:
import gurobipy as gp
from gurobipy import *
from gurobipy import GRB
import numpy as np
# import scipy.sparse as sp

# Classes

In [46]:
class Dia:
    def __init__(self, id: int, nombre: str):
        self.id = id
        self.nombre = nombre

    def __str__(self) -> str:
        return self.nombre
        # return self.id
    
    def __eq__(self, __value: object) -> bool:
        return type(__value) == type(self) and self.nombre == __value.nombre
 
class Horario:
    def __init__(self, id: int, inicio: str, fin: str):
        self.id = id
        self.inicio = inicio
        self.fin = fin

    def __str__(self) -> str:
        return self.inicio + "-" + self.fin
        # return self.id
    
    def __eq__(self, __value: object) -> bool:
        return type(__value) == type(self) and self.inicio == __value.inicio and self.fin == __value.fin
    
class BloqueHorario:
    def __init__(self, dia: Dia, horario: Horario) -> None:
        self.dia = dia
        self.horario = horario
        
    def __str__(self) -> str:
        return str(self.dia) + "_" + str(self.horario)
    
    def __eq__(self, __value: object) -> bool:
        return type(__value) == type(self) and self.dia == __value.dia and self.horario == __value.horario
    
    def id(self):
        return (self.dia.id, self.horario.id)
      
# ver disponibilidad
class Profesor:
    def __init__(self, id: int, nombre: str):
        self.id = id
        self.nombre = nombre
        self.no_disponible = []
        self.prioridades = []

    def __str__(self) -> str:
        return self.nombre
    
    def __eq__(self, __value: object) -> bool:
        return type(__value) == type(self) and self.id == __value.id
    
    # def disponibilidad(self, bloques_horario):
    #     disp = bloques_horario.copy()
    #     for d in self.no_disponible:
    #         disp.pop((d.dia.id, d.horario.id))
    #     return disp
        

class Materia:
    def __init__(self,
                 id: int,
                 nombre: str,
                 profesor: Profesor=None,
                 carga_horaria: int=None,
                 cantidad_dias: int=3,
                 horas_maximo: int=10,
                 horas_minimo: int=0,
                 anio: int=None,
                 grupo: int=None,
                 lista_profesores: [Profesor] = []
                 ) -> None:
        
        self.nombre = nombre
        self.id = id
        self.carga_horaria = carga_horaria  #C_m
        self.profesor = profesor
        self.cantidad_dias = cantidad_dias  #D_m
        self.horas_maximo = horas_maximo    # Hmax_m
        self.horas_minimo = horas_minimo    # Hmin_m
        self.anio = anio
        self.grupo = grupo
        self.lista_profesores = lista_profesores
        
    def __str__(self) -> str:
        if self.grupo is not None:
            return self.nombre + str(self.grupo)
        return self.nombre
    
    def __eq__(self, __value: object) -> bool:
        return type(__value) == type(self) and self.id == __value.id

class Prioridad:
    def __init__(self, value: int, profesor: Profesor, bloque_horario: BloqueHorario) -> None:
        self.value = value
        self.profesor = profesor
        self.bloque_horario = bloque_horario
    
    def __str__(self) -> str:
        return str(self.profesor) + "_" + str(self.bloque_horario) + ":prioridad=" + str(self.value)
    
    def __eq__(self, __value: object) -> bool:
        return type(__value) == type(self) and self.profesor == __value.profesor and self.bloque_horario == __value.bloque_horario
    
    def id(self):
        return (self.profesor.id, self.bloque_horario.id())


In [47]:
# variables    
class u:
    def __init__(self, materia: Materia, horario: BloqueHorario) -> None:
        self.materia = materia
        self.horario = horario
        self.variable = None

    def __str__(self) -> str:
        return "u_{" + str(self.materia) + "_" + str(self.horario) + "}"
    
    def __eq__(self, __value: object) -> bool:
        return type(__value) == type(self) and self.materia == __value.materia and self.horario == __value.horario
    
class v:
    def __init__(self, materia: Materia, dia: Dia) -> None:
        self.materia = materia
        self.dia = dia
        self.variable = None

    def __str__(self) -> str:
        return "v_{" + str(self.materia) + "_" + str(self.dia) + "}"
    
    def __eq__(self, __value: object) -> bool:
        return type(__value) == type(self) and self.materia == __value.materia and self.dia == __value.dia

class w:
    def __init__(self, materia: Materia, profesor: Profesor) -> None:
        self.materia = materia
        self.profesor = profesor
        self.variable = None
    
    def __str__(self) -> str:
        return "w_{" + str(self.materia) + "_" + str(self.profesor) + "}"
    
    def __eq__(self, __value: object) -> bool:
        return type(__value) == type(self) and self.profesor == __value.profesor and self.materia == __value.materia
    


# opcional    
class r:
    def __init__(self, profesor: Profesor) -> None:
        self.profesor = profesor
        self.variable = None
    
    def __str__(self) -> str:
        return "r_{" + str(self.profesor) + "}"
    
    def __eq__(self, __value: object) -> bool:
        return type(__value) == type(self) and self.profesor == __value.profesor
    

# Functions

In [48]:
#prioridades
def update_no_disp(profesor, bloques_horario, no_disp_index):
    for i in no_disp_index:
        if bloques_horario[i] not in profesor.no_disponible:
            profesor.no_disponible.append(bloques_horario[i])

def update_prioridad(profesor, bloques_horario, array_prioridad):
    # array_prioridad[i] = [(d,h),a]
    for i in array_prioridad:
        prior = Prioridad(i[1], profesor, bloques_horario[i[0]])
        if not (prior.bloque_horario in profesor.no_disponible or prior in profesor.prioridades):
            profesor.prioridades.append(prior)


In [49]:
#imprimir
def print_timetable(dias, horarios, u_dict, w_dict, grupos = None):
    for g in grupos:
        print('\n', "Grupo: ", str(g))
        print('\t', *[str(d) for d in dias], sep='\t')
        print('·····················································')
        for h in horarios:
            mats = [search_materia(BloqueHorario(d,h), g, u_dict) for d in dias]
            lista = []
            for m in mats:
                if m is None:
                    lista.append("---")
                else:
                    p = search_profesor(m, w_dict)
                    lista.append(str(m) + " (" + str(p) + ")")
            print(str(h), *lista, sep='\t')

def print_prof_timetable(dias, horarios, u_dict, w_dict, profesores):
    for p in profesores:
        print('\n', "Profesor: ", str(p))
        print('\t', *[str(d) for d in dias], sep='\t')
        print('·····················································')
        for h in horarios:
            mats = [search_materia_prof(BloqueHorario(d,h), p, u_dict, w_dict) for d in dias]
            lista = []
            for m in mats:
                if m is None:
                    lista.append("---")
                else:
                    lista.append(str(m))
            print(str(h), *lista, sep='\t')

def search_materia(b, g, u_dict):
    for ui in u_dict:
        u_ob = u_dict[ui]
        if u_ob.horario == b and u_ob.materia.grupo == g and round(u_ob.variable.x) == 1:
            return u_ob.materia
    return None

def search_profesor(materia, w_dict):
    if materia is not None:
        if materia.profesor is not None:
            return materia.profesor
        else:
            for p in materia.lista_profesores:
                if round(w_dict[materia.id, p.id].variable.x) == 1:
                    return p
    else:
        return None
    
def search_materia_prof(b, p, u_dict, w_dict):
    for ui in u_dict:
        u_ob = u_dict[ui]
        bloque = u_ob.horario
        profesor = search_profesor(u_ob.materia, w_dict)
        if bloque == b and profesor == p and round(u_ob.variable.x) == 1:
            return u_ob.materia
    return None


In [50]:
#buscar materias
def materias_fijas_profesor(profesor, materias_total):
    materias = []

    for m in materias_total:
        if m.profesor == profesor and profesor is not None:
            materias.append(m)

    return materias

def materias_variables_profesor(profesor, materias_total):
    materias = []

    for m in materias_total:
        if m.profesor is None and profesor in m.lista_profesores:
            materias.append(m)
            
    return materias

def materias_grupo(grupo, materias_total):
    materias = []

    for m in materias_total:
        if m.grupo == grupo and grupo is not None:
            materias.append(m)
            
    return materias


# Formulacion

## (1) Constantes / Datos

In [51]:
dias = [
    Dia(1, "lun"),
    Dia(2, "mar"),
    Dia(3, "mie"),
    Dia(4, "jue"),
    Dia(5, "vie")
]
dias_ids = []
for d in dias:
    dias_ids.append(d.id)

horarios = [
    Horario(1, "8:00", "8:50"),
    Horario(2, "8:50", "9:40"),
    Horario(3, "9:50", "10:40"),
    Horario(4, "10:40", "11:30"),
    Horario(5, "11:40", "12:30"),
    Horario(6, "12:30", "13:20")
]
horarios_ids = []
for h in horarios:
    horarios_ids.append(h.id)


bloques_horario = {}
for dia in dias:
    for horario in horarios:
        bloques_horario[(dia.id, horario.id)] = BloqueHorario(dia, horario)

bloques_horario_ids = []
for b in bloques_horario:
    bloques_horario_ids.append(b)


In [52]:
profesores = [
    Profesor(0, "a"),
    Profesor(1, "b"),
    Profesor(2, "c"),
    Profesor(3, "d"),
    Profesor(4, "e"),
    Profesor(5, "f"),
    Profesor(6, "g"),
    Profesor(7, "h"),
    Profesor(8, "i"),
    Profesor(9, "j"),
    Profesor(10, "k"),
    Profesor(11, "l")
]
profesores_ids = []
for p in profesores:
    profesores_ids.append(p.id)



In [53]:
#prioridad
update_prioridad(profesores[0], bloques_horario, [[(1,3),1],
                                                  [(1,4),1],
                                                  [(2,3),1],
                                                  [(2,4),1],
                                                  [(2,5),3],
                                                  [(2,6),3],
                                                  [(3,1),1],
                                                  [(3,2),1],
                                                  [(4,3),2],
                                                  [(4,4),2],
                                                  [(4,5),2],
                                                  [(5,3),1],
                                                  [(5,4),1],
                                                  [(5,5),3],
                                                  [(5,6),3]
                                                  ])

update_prioridad(profesores[1], bloques_horario, [[(1,1),1],
                                                  [(1,2),1],
                                                  [(1,3),1],
                                                  [(1,4),1],
                                                  [(2,3),3],
                                                  [(2,4),3],
                                                  [(3,1),2],
                                                  [(3,2),2],
                                                  [(3,3),2],
                                                  [(3,4),2],
                                                  [(4,3),1],
                                                  [(4,4),1],
                                                  [(4,5),1],
                                                  [(4,6),1],
                                                  [(5,3),3],
                                                  [(5,4),3],
                                                  [(5,5),1],
                                                  [(5,6),1],
                                                 ])

update_prioridad(profesores[2], bloques_horario, [[(1,3),1],
                                                  [(1,4),1],
                                                  [(1,5),3],
                                                  [(1,6),3],
                                                  [(2,1),1],
                                                  [(2,2),1],
                                                  [(2,3),2],
                                                  [(2,4),2],
                                                  [(2,5),1],
                                                  [(2,6),1],
                                                  [(3,3),3],
                                                  [(3,4),3],
                                                  [(3,5),2],
                                                  [(3,6),2],
                                                  [(4,3),1],
                                                  [(4,4),1],
                                                  [(4,5),1],
                                                  [(4,6),1],
                                                  [(5,3),2],
                                                  [(5,4),2],
                                                  [(5,5),3],
                                                  [(5,6),3],
                                                  ])

update_prioridad(profesores[3], bloques_horario, [[(1,1),3],
                                                  [(1,2),3],
                                                  [(1,3),1],
                                                  [(1,4),1],
                                                  [(2,1),1],
                                                  [(2,2),1],
                                                  [(2,5),1],
                                                  [(2,6),1],
                                                  [(3,1),2],
                                                  [(3,2),2],
                                                  [(3,5),1],
                                                  [(3,6),1],
                                                  [(4,1),3],
                                                  [(4,2),3],
                                                  [(4,5),1],
                                                  [(4,6),1],
                                                  [(5,1),1],
                                                  [(5,2),1],
                                                  [(5,3),1],
                                                  [(5,4),1],
                                                  ])

update_prioridad(profesores[4], bloques_horario, [[(1,3),1],
                                                  [(1,4),1],
                                                  [(2,3),1],
                                                  [(2,4),1],
                                                  [(2,5),3],
                                                  [(2,6),3],
                                                  [(3,1),1],
                                                  [(3,2),1],
                                                  [(4,3),2],
                                                  [(4,4),2],
                                                  [(4,5),2],
                                                  [(5,3),1],
                                                  [(5,4),1],
                                                  [(5,5),3],
                                                  [(5,6),3]
                                                  ])

update_prioridad(profesores[5], bloques_horario, [[(1,1),1],
                                                  [(1,2),1],
                                                  [(1,3),1],
                                                  [(1,4),1],
                                                  [(2,3),3],
                                                  [(2,4),3],
                                                  [(3,1),2],
                                                  [(3,2),2],
                                                  [(3,3),2],
                                                  [(3,4),2],
                                                  [(4,3),1],
                                                  [(4,4),1],
                                                  [(4,5),1],
                                                  [(4,6),1],
                                                  [(5,3),3],
                                                  [(5,4),3],
                                                  [(5,5),1],
                                                  [(5,6),1],
                                                 ])

update_prioridad(profesores[6], bloques_horario, [[(1,3),1],
                                                  [(1,4),1],
                                                  [(1,5),3],
                                                  [(1,6),3],
                                                  [(2,1),1],
                                                  [(2,2),1],
                                                  [(2,3),2],
                                                  [(2,4),2],
                                                  [(2,5),1],
                                                  [(2,6),1],
                                                  [(3,3),3],
                                                  [(3,4),3],
                                                  [(3,5),2],
                                                  [(3,6),2],
                                                  [(4,3),1],
                                                  [(4,4),1],
                                                  [(4,5),1],
                                                  [(4,6),1],
                                                  [(5,3),2],
                                                  [(5,4),2],
                                                  [(5,5),3],
                                                  [(5,6),3],
                                                  ])

update_prioridad(profesores[7], bloques_horario, [[(1,1),3],
                                                  [(1,2),3],
                                                  [(1,3),1],
                                                  [(1,4),1],
                                                  [(2,1),1],
                                                  [(2,2),1],
                                                  [(2,5),1],
                                                  [(2,6),1],
                                                  [(3,1),2],
                                                  [(3,2),2],
                                                  [(3,5),1],
                                                  [(3,6),1],
                                                  [(4,1),3],
                                                  [(4,2),3],
                                                  [(4,5),1],
                                                  [(4,6),1],
                                                  [(5,1),1],
                                                  [(5,2),1],
                                                  [(5,3),1],
                                                  [(5,4),1],
                                                  ])

update_prioridad(profesores[8], bloques_horario, [[(1,3),1],
                                                  [(1,4),1],
                                                  [(2,3),1],
                                                  [(2,4),1],
                                                  [(2,5),3],
                                                  [(2,6),3],
                                                  [(3,1),1],
                                                  [(3,2),1],
                                                  [(4,3),2],
                                                  [(4,4),2],
                                                  [(4,5),2],
                                                  [(5,3),1],
                                                  [(5,4),1],
                                                  [(5,5),3],
                                                  [(5,6),3]
                                                  ])

update_prioridad(profesores[9], bloques_horario, [[(1,1),1],
                                                  [(1,2),1],
                                                  [(1,3),1],
                                                  [(1,4),1],
                                                  [(2,3),3],
                                                  [(2,4),3],
                                                  [(3,1),2],
                                                  [(3,2),2],
                                                  [(3,3),2],
                                                  [(3,4),2],
                                                  [(4,3),1],
                                                  [(4,4),1],
                                                  [(4,5),1],
                                                  [(4,6),1],
                                                  [(5,3),3],
                                                  [(5,4),3],
                                                  [(5,5),1],
                                                  [(5,6),1],
                                                 ])

update_prioridad(profesores[10], bloques_horario, [[(1,3),1],
                                                  [(1,4),1],
                                                  [(1,5),3],
                                                  [(1,6),3],
                                                  [(2,1),1],
                                                  [(2,2),1],
                                                  [(2,3),2],
                                                  [(2,4),2],
                                                  [(2,5),1],
                                                  [(2,6),1],
                                                  [(3,3),3],
                                                  [(3,4),3],
                                                  [(3,5),2],
                                                  [(3,6),2],
                                                  [(4,3),1],
                                                  [(4,4),1],
                                                  [(4,5),1],
                                                  [(4,6),1],
                                                  [(5,3),2],
                                                  [(5,4),2],
                                                  [(5,5),3],
                                                  [(5,6),3],
                                                  ])

update_prioridad(profesores[11], bloques_horario, [[(1,1),3],
                                                  [(1,2),3],
                                                  [(1,3),1],
                                                  [(1,4),1],
                                                  [(2,1),1],
                                                  [(2,2),1],
                                                  [(2,5),1],
                                                  [(2,6),1],
                                                  [(3,1),2],
                                                  [(3,2),2],
                                                  [(3,5),1],
                                                  [(3,6),1],
                                                  [(4,1),3],
                                                  [(4,2),3],
                                                  [(4,5),1],
                                                  [(4,6),1],
                                                  [(5,1),1],
                                                  [(5,2),1],
                                                  [(5,3),1],
                                                  [(5,4),1],
                                                  ])

# no disponibilidad
for p in profesores:

    no_disp_index = bloques_horario.copy()
    for pr in p.prioridades:
        no_disp_index.pop((pr.bloque_horario.id()))

    update_no_disp(p, bloques_horario, no_disp_index)


#disponibilidad
# update_no_disp(profesores[0], bloques_horario, [(1,1),(1,2),(1,5),(1,6),(2,1),(2,2),(3,3),(3,4),(3,5),(3,6),(4,1),(4,2),(4,6),(5,1),(5,2)])
# update_no_disp(profesores[1], bloques_horario, [(1,5),(1,6),(2,1),(2,2),(2,5),(2,6),(3,5),(3,6),(4,1),(4,2),(5,1),(5,2)])
# update_no_disp(profesores[2], bloques_horario, [(1,1),(1,2),(3,1),(3,2),(4,1),(4,2),(5,1),(5,2)])
# update_no_disp(profesores[3], bloques_horario, [(1,5),(1,6),(2,3),(2,4),(3,3),(3,4),(4,3),(4,4),(5,5),(5,6)])
# update_no_disp(profesores[4], bloques_horario, [(1,1),(1,2),(1,5),(1,6),(2,1),(2,2),(3,3),(3,4),(3,5),(3,6),(4,1),(4,2),(4,6),(5,1),(5,2)])
# update_no_disp(profesores[5], bloques_horario, [(1,5),(1,6),(2,1),(2,2),(2,5),(2,6),(3,5),(3,6),(4,1),(4,2),(5,1),(5,2)])
# update_no_disp(profesores[6], bloques_horario, [(1,1),(1,2),(3,1),(3,2),(4,1),(4,2),(5,1),(5,2)])
# update_no_disp(profesores[7], bloques_horario, [(1,5),(1,6),(2,3),(2,4),(3,3),(3,4),(4,3),(4,4),(5,5),(5,6)])
# update_no_disp(profesores[8], bloques_horario, [(1,1),(1,2),(1,5),(1,6),(2,1),(2,2),(3,3),(3,4),(3,5),(3,6),(4,1),(4,2),(4,6),(5,1),(5,2)])
# update_no_disp(profesores[9], bloques_horario, [(1,5),(1,6),(2,1),(2,2),(2,5),(2,6),(3,5),(3,6),(4,1),(4,2),(5,1),(5,2)])
# update_no_disp(profesores[10], bloques_horario, [(1,1),(1,2),(3,1),(3,2),(4,1),(4,2),(5,1),(5,2)])
# update_no_disp(profesores[11], bloques_horario, [(1,5),(1,6),(2,3),(2,4),(3,3),(3,4),(4,3),(4,4),(5,5),(5,6)])


# for p in profesores:
#     print(str(p))
#     print([str(d) for d in p.no_disponible])


In [54]:
#materias

## grupo 1
materias = []
materias.append(Materia(0, "A", profesores[0], carga_horaria=5, cantidad_dias=2, horas_maximo=3))
materias.append(Materia(1, "B", profesores[1], carga_horaria=7, cantidad_dias=3, horas_maximo=3, horas_minimo=2))
materias.append(Materia(2, "C", profesores[2], carga_horaria=7, cantidad_dias=3, horas_maximo=3, horas_minimo=2))
materias.append(Materia(3, "D", profesores[3], carga_horaria=5, cantidad_dias=2, horas_maximo=3))
for m in materias:
    m.grupo = 1

n = len(materias)
## grupo 2 (mismas materias)
for i in range(n):
    m = materias[i]
    materias.append(Materia(i+n, m.nombre,
                            carga_horaria=m.carga_horaria,
                            cantidad_dias=m.cantidad_dias,
                            horas_maximo=m.horas_maximo,
                            horas_minimo=m.horas_minimo,
                            grupo=2,
                            lista_profesores=[p for p in profesores[0:8]]))
                            ## asignacion variable de profesores

## grupo 3 (mismas materias)
for i in range(n):
    m = materias[i]
    materias.append(Materia(i+2*n, m.nombre, profesores[i+n],
                            carga_horaria=m.carga_horaria,
                            cantidad_dias=m.cantidad_dias,
                            horas_maximo=m.horas_maximo,
                            horas_minimo=m.horas_minimo,
                            grupo=3))

materias_ids = []
grupos = []
for m in materias:
    materias_ids.append(m.id)
    if m.grupo is not None and m.grupo not in grupos:
        grupos.append(m.grupo)
    print(m)

print(grupos)


A1
B1
C1
D1
A2
B2
C2
D2
A3
B3
C3
D3
[1, 2, 3]


## (2) Variables


$$ u_{mb} = [0,1] $$

    m: materia
    b: bloque horario = [dh]: dia y hora

$$ v_{md} = [0,1] $$

    m: materia
    d: dia

$$ r_p = \R $$

    p: profesor

$$ w_{mp} = [0,1] $$

    m: materia
    p: profesor


In [55]:
# variables

u_dict = {}
for m in materias:        #crear variables "u" solo para horarios disponibles
    # disp = m.profesor.disponibilidad(bloques_horario)
    for b_id in bloques_horario:
        u_dict[(m.id, b_id)] = u(m, bloques_horario[b_id])
print("u: ", len(u_dict))

# print([str(us) for us in u_dict])

v_dict = {}
for m in materias:
    for d in dias:
        v_dict[(m.id, d.id)] = v(m, d)
print("v: ", len(v_dict))


# opcional
# r_dict = {}
# for p in profesores:
#     r_dict[(p.id)] = r(p)

w_dict = {}
for m in materias:
    if m.profesor is None:
        for p in m.lista_profesores:
            w_dict[(m.id, p.id)] = w(m, p)
print("w: ", len(w_dict))


u:  360
v:  60
w:  32


In [56]:
# Create a new model
model = gp.Model("timetable")


In [57]:
# Create variables

# u_vars = m.addMVar(shape=len(u_dict), vtype=GRB.BINARY, name="u") # variable matrix

for u_i in u_dict: # crear variables "u" a partir de u_dict
    u_dict[u_i].variable = model.addVar(vtype=GRB.BINARY, name=str(u_dict[u_i]))

# print([ui.variable for ui in u_dict])

for v_i in v_dict: # crear variables "v" a partir de v_dict
    v_dict[v_i].variable = model.addVar(vtype=GRB.BINARY, name=str(v_dict[v_i]))

# v_vars = np.array([v_i.variable for v_i in v_dict])

# for r_i in r_dict:
#     r_dict[r_i].variable = model.addVar(vtype=GRB.CONTINUOUS, name=str(r_dict[r_i]))

for w_i in w_dict: # crear variables "v" a partir de v_dict
    w_dict[w_i].variable = model.addVar(vtype=GRB.BINARY, name=str(w_dict[w_i]))



## (3) Restricciones

### (3.1) unica materia por bloque horario por grupo
$$ \sum_{m.grupo = g}{u_{mb}} \leq 1 $$

    para todo b, g

In [58]:
for g in grupos:
    mats = materias_grupo(g, materias)
    if len(mats) > 1:
        model.addConstrs(gp.quicksum(u_dict[m.id, b].variable for m in mats) <= 1 for b in bloques_horario_ids)

### (3.2) cubrir carga horaria para cada materia
$$ \sum_b{u_{mb}} = c_m $$

    para todo m (c_m: carga horaria)

In [59]:
model.addConstrs(gp.quicksum(u_dict[m, b].variable for b in bloques_horario_ids) == materias[m].carga_horaria for m in materias_ids)

{0: <gurobi.Constr *Awaiting Model Update*>,
 1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>,
 3: <gurobi.Constr *Awaiting Model Update*>,
 4: <gurobi.Constr *Awaiting Model Update*>,
 5: <gurobi.Constr *Awaiting Model Update*>,
 6: <gurobi.Constr *Awaiting Model Update*>,
 7: <gurobi.Constr *Awaiting Model Update*>,
 8: <gurobi.Constr *Awaiting Model Update*>,
 9: <gurobi.Constr *Awaiting Model Update*>,
 10: <gurobi.Constr *Awaiting Model Update*>,
 11: <gurobi.Constr *Awaiting Model Update*>}

### (3.3) definicion de "v"

$$ v_{md} = {OR}_h [u_{mdh}] $$
    para todo m, d

In [60]:
model.addConstrs(v_dict[(m, d)].variable == gp.or_(u_dict[(m, (d, h))].variable for h in horarios_ids)  for m in materias_ids for d in dias_ids)

{(0, 1): <gurobi.GenConstr *Awaiting Model Update*>,
 (0, 2): <gurobi.GenConstr *Awaiting Model Update*>,
 (0, 3): <gurobi.GenConstr *Awaiting Model Update*>,
 (0, 4): <gurobi.GenConstr *Awaiting Model Update*>,
 (0, 5): <gurobi.GenConstr *Awaiting Model Update*>,
 (1, 1): <gurobi.GenConstr *Awaiting Model Update*>,
 (1, 2): <gurobi.GenConstr *Awaiting Model Update*>,
 (1, 3): <gurobi.GenConstr *Awaiting Model Update*>,
 (1, 4): <gurobi.GenConstr *Awaiting Model Update*>,
 (1, 5): <gurobi.GenConstr *Awaiting Model Update*>,
 (2, 1): <gurobi.GenConstr *Awaiting Model Update*>,
 (2, 2): <gurobi.GenConstr *Awaiting Model Update*>,
 (2, 3): <gurobi.GenConstr *Awaiting Model Update*>,
 (2, 4): <gurobi.GenConstr *Awaiting Model Update*>,
 (2, 5): <gurobi.GenConstr *Awaiting Model Update*>,
 (3, 1): <gurobi.GenConstr *Awaiting Model Update*>,
 (3, 2): <gurobi.GenConstr *Awaiting Model Update*>,
 (3, 3): <gurobi.GenConstr *Awaiting Model Update*>,
 (3, 4): <gurobi.GenConstr *Awaiting Model Upd

### (3.4) particion de horas por materia

#### (3.4.1) fijar cantidad de dias por materia

$$ \sum_d {v_{md}} = D_m $$

    para todo m

In [61]:
model.addConstrs(gp.quicksum(v_dict[m, d].variable for d in dias_ids) == materias[m].cantidad_dias for m in materias_ids)

{0: <gurobi.Constr *Awaiting Model Update*>,
 1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>,
 3: <gurobi.Constr *Awaiting Model Update*>,
 4: <gurobi.Constr *Awaiting Model Update*>,
 5: <gurobi.Constr *Awaiting Model Update*>,
 6: <gurobi.Constr *Awaiting Model Update*>,
 7: <gurobi.Constr *Awaiting Model Update*>,
 8: <gurobi.Constr *Awaiting Model Update*>,
 9: <gurobi.Constr *Awaiting Model Update*>,
 10: <gurobi.Constr *Awaiting Model Update*>,
 11: <gurobi.Constr *Awaiting Model Update*>}

#### (3.4.2) fijar maximo y minimo de horas por dia

$$ v_{md} \times Hmin_m \leq \sum_h {u_{mdh}} \leq Hmax_m $$

    para todo m, d

In [62]:
model.addConstrs(gp.quicksum(u_dict[(m,(d,h))].variable for h in horarios_ids)
                 <= materias[m].horas_maximo for m in materias_ids for d in dias_ids)

model.addConstrs(gp.quicksum(u_dict[(m,(d,h))].variable for h in horarios_ids)
                 >= materias[m].horas_minimo * v_dict[m,d].variable for m in materias_ids for d in dias_ids)

{(0, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 2): <gurobi.Constr *Awaiting Model Update*>,
 (0, 3): <gurobi.Constr *Awaiting Model Update*>,
 (0, 4): <gurobi.Constr *Awaiting Model Update*>,
 (0, 5): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1): <gurobi.Constr *Awaiting Model Update*>,
 (1, 2): <gurobi.Constr *Awaiting Model Update*>,
 (1, 3): <gurobi.Constr *Awaiting Model Update*>,
 (1, 4): <gurobi.Constr *Awaiting Model Update*>,
 (1, 5): <gurobi.Constr *Awaiting Model Update*>,
 (2, 1): <gurobi.Constr *Awaiting Model Update*>,
 (2, 2): <gurobi.Constr *Awaiting Model Update*>,
 (2, 3): <gurobi.Constr *Awaiting Model Update*>,
 (2, 4): <gurobi.Constr *Awaiting Model Update*>,
 (2, 5): <gurobi.Constr *Awaiting Model Update*>,
 (3, 1): <gurobi.Constr *Awaiting Model Update*>,
 (3, 2): <gurobi.Constr *Awaiting Model Update*>,
 (3, 3): <gurobi.Constr *Awaiting Model Update*>,
 (3, 4): <gurobi.Constr *Awaiting Model Update*>,
 (3, 5): <gurobi.Constr *Awaiting Model Update*>,


### (3.5) horas consecutivas dentro de un dia

$$ u_{mdh} + \sum_h {u_{md(h)} \otimes u_{md(h+1)}} + u_{md(h_{max})} = 2 $$

$$ \sum_h {u_{mdh}} - \sum_h {u_{md(h)} · u_{md(h+1)}} = 1 $$

$$ \sum_h {u_{mdh}} - \sum_h {u_{md(h)} · u_{md(h+1)}} = v_{md} $$
    para todo m, d

In [63]:
model.addConstrs(gp.quicksum(u_dict[(m,(d,h))].variable for h in horarios_ids)
                 - gp.quicksum(u_dict[(m,(d,h))].variable * u_dict[(m,(d,h+1))].variable for h in horarios_ids[0:len(horarios_ids)-1])
                 == v_dict[m, d].variable for m in materias_ids for d in dias_ids)        

{(0, 1): <gurobi.QConstr Not Yet Added>,
 (0, 2): <gurobi.QConstr Not Yet Added>,
 (0, 3): <gurobi.QConstr Not Yet Added>,
 (0, 4): <gurobi.QConstr Not Yet Added>,
 (0, 5): <gurobi.QConstr Not Yet Added>,
 (1, 1): <gurobi.QConstr Not Yet Added>,
 (1, 2): <gurobi.QConstr Not Yet Added>,
 (1, 3): <gurobi.QConstr Not Yet Added>,
 (1, 4): <gurobi.QConstr Not Yet Added>,
 (1, 5): <gurobi.QConstr Not Yet Added>,
 (2, 1): <gurobi.QConstr Not Yet Added>,
 (2, 2): <gurobi.QConstr Not Yet Added>,
 (2, 3): <gurobi.QConstr Not Yet Added>,
 (2, 4): <gurobi.QConstr Not Yet Added>,
 (2, 5): <gurobi.QConstr Not Yet Added>,
 (3, 1): <gurobi.QConstr Not Yet Added>,
 (3, 2): <gurobi.QConstr Not Yet Added>,
 (3, 3): <gurobi.QConstr Not Yet Added>,
 (3, 4): <gurobi.QConstr Not Yet Added>,
 (3, 5): <gurobi.QConstr Not Yet Added>,
 (4, 1): <gurobi.QConstr Not Yet Added>,
 (4, 2): <gurobi.QConstr Not Yet Added>,
 (4, 3): <gurobi.QConstr Not Yet Added>,
 (4, 4): <gurobi.QConstr Not Yet Added>,
 (4, 5): <gurobi

In [64]:
abc = {1:300, 2:250, 3:200}
abc = {i:i**2 for i in range(5)}
abc = {"a":2, "b":3}
abc = {("a","a"):1, ("a","b"):3}
abc[("b","b")] = 5
t = ("b","b")
# try:
#     print(abc[t])
# except:
#     pass




### (3.6) indisponibilidad

$$ u_{mb} = 0 $$
$$ w_{mp} \times u_{mb} = 0 $$

    para todo p sin disponibilidad en b

In [65]:
for m in materias:
    if m.profesor is not None:
        for b in m.profesor.no_disponible:
            model.addConstr(u_dict[m.id, b.id()].variable == 0)
    else:
        for p in m.lista_profesores:
            for b in p.no_disponible:
                model.addConstr(u_dict[m.id, b.id()].variable * w_dict[m.id, p.id].variable == 0)


### (3.7) unica materia por profesor para un mismo bloque horario

$$ \sum_{m.prof = p} {u_{mb}} + \sum_{m} {w_{mp} \times u_{mb}} \leq 1 $$

    para todo b, p

In [66]:
# model.addConstrs(gp.quicksum(u_dict[m.id, b].variable for m in materias_fijas_profesor(profesores[p-1], materias)) <= 1 for b in bloques_horario_ids for p in profesores_ids)

# solo asigancion fija:
# for p in profesores:
#     mats = materias_fijas_profesor(p, materias)
    
#     if len(mats) > 1:
#         model.addConstrs(gp.quicksum(u_dict[m.id, b].variable for m in mats) <= 1 for b in bloques_horario_ids)

for p in profesores:
    f_mats = materias_fijas_profesor(p, materias) # materias con el profesor pre-asignado
    var_mats = materias_variables_profesor(p, materias) # materias sin el profesor asignado
    
    if len(f_mats) + len(var_mats) > 1:
        model.addConstrs(gp.quicksum(u_dict[m.id, b].variable for m in f_mats) +
                         gp.quicksum(u_dict[m.id, b].variable * w_dict[m.id, p.id].variable for m in var_mats)
                          <= 1 for b in bloques_horario_ids)

### (3.8) unico profesor por materia

$$ \sum_p {w_{mp}} = 1 $$

para todo m

In [67]:
for m in materias:
    if m.profesor is None:
        model.addConstr(gp.quicksum(w_dict[m.id, p.id].variable for p in m.lista_profesores) == 1)

### (3.9) definicion de rating r_p

opcional

$$ r_p = \frac{\sum_{b, m.prof=p} {A_{mb} · u_{mb}}}{\sum_{m.prof=p}{C_m}} $$
$$ r_p \sum_{m.prof=p}{C_m} = \sum_{b, m.prof=p} {A_{mb} · u_{mb}}  $$

In [68]:
# for p in profesores:
#     r_coef = 0
#     sum_expr = gp.LinExpr()
#     for m in materias_fijas_profesor(p, materias):
#         r_coef += m.carga_horaria
#         for pr in p.prioridades:
#             sum_expr += pr.value * u_dict[m.id, pr.bloque_horario.id()].variable

#     model.addConstr(r_coef * r_dict[p.id].variable == sum_expr)

### (3.10) carga horaria por docente: limitar la cantidad de materias por profesor

$$ \sum_m {w_{mp}} = K_p $$

    para todo p

In [69]:
for p in profesores:
    var_mats = materias_variables_profesor(p, materias) # materias sin el profesor asignado
    if len(var_mats) > 1:
        model.addConstr(gp.quicksum(w_dict[m.id, p.id].variable for m in var_mats) <= 1)

## (4) Funcion Objetivo

In [70]:
# Set objective
obj = gp.QuadExpr()

### (4.1) Prioridad horaria de los docentes

$$ \sum_{m,b} {A_{mb}·u_{mb}} + \sum_{m,b} {A_{mb}·w_{mp}·u_{mb}} $$
alternativa:
$$ \sum_p {r_p}

In [71]:
for ui in u_dict:
    ai = None
    u_ob = u_dict[ui]
    materia = u_ob.materia
    
    profesor = materia.profesor
    if profesor is not None:
        for pr in profesor.prioridades:
            if pr.bloque_horario == u_ob.horario:
                ai = pr
                break
        if ai is not None:
            obj += ai.value * u_ob.variable

    else:
        for p in materia.lista_profesores:
            for pr in p.prioridades:
                if pr.bloque_horario == u_ob.horario:
                    ai = pr
                    break
            if ai is not None:
                obj += ai.value * u_ob.variable * w_dict[materia.id, p.id].variable


# alternativa: ratings
# obj += gp.quicksum(r_dict[p].variable for p in profesores_ids)



### (4.2) Minimizar dias consecutivos con la misma materia

$$ \sum_m {\sum_d {v_{m(d)} · v_{m(d+1)}}} $$

In [72]:
PESO = 10

for m in materias_ids:
    obj += PESO * gp.quicksum(v_dict[m,d].variable * v_dict[m,d+1].variable for d in dias_ids[0:len(dias_ids)-1])

### (4.4) Maximizar balance global de felicidad (rating): minimizar varianza de los r_p
opcional

$$ \frac{1}{P} \sum_p {({r_p - \frac{1}{P} \sum_{p'} {r_{p'}}})^2} $$

In [73]:
# PESO = 50
# m = np.mean([r_dict[ri].variable for ri in r_dict])
# d2 = np.mean([(r_dict[ri].variable - m)**2 for ri in r_dict])
# obj += PESO*d2

## Resolver

In [74]:
model.setObjective(obj, GRB.MINIMIZE)
model.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11.0 (22621.2))

CPU model: Intel(R) Core(TM) i7-9750H CPU @ 2.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 336 rows, 452 columns and 1684 nonzeros
Model fingerprint: 0xbc3b1fb9
Model has 892 quadratic objective terms
Model has 660 quadratic constraints
Model has 60 general constraints
Variable types: 0 continuous, 452 integer (452 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 1e+00]
  Objective range  [1e+00, 3e+00]
  QObjective range [2e+00, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 7e+00]
  QRHS range       [1e+00, 1e+00]
Presolve added 256 rows and 0 columns
Presolve removed 0 rows and 94 columns
Presolve time: 0.01s
Presolved: 2474 rows, 1560 columns, 8216 nonzeros
Variable types: 0 continuous, 1560 integer (1560 bina

In [75]:
# result = np.array([var.x for var in model.getVars()])
# print(result)
# print(model.getVars())
# for v in model.getVars():
#     print('%s %g' % (v.VarName, v.X))

print('Obj: %g' % model.ObjVal)
print('=============================================================')
print_timetable(dias, horarios, u_dict, w_dict, grupos)


Obj: 149

 Grupo:  1
		lun	mar	mie	jue	vie
·····················································
8:00-8:50	B1 (b)	C1 (c)	---	---	D1 (d)
8:50-9:40	B1 (b)	C1 (c)	B1 (b)	---	D1 (d)
9:50-10:40	C1 (c)	A1 (a)	B1 (b)	A1 (a)	D1 (d)
10:40-11:30	C1 (c)	A1 (a)	B1 (b)	A1 (a)	---
11:40-12:30	C1 (c)	A1 (a)	D1 (d)	C1 (c)	B1 (b)
12:30-13:20	---	---	D1 (d)	C1 (c)	B1 (b)

 Grupo:  2
		lun	mar	mie	jue	vie
·····················································
8:00-8:50	---	C2 (g)	D2 (a)	B2 (h)	---
8:50-9:40	B2 (h)	C2 (g)	D2 (a)	B2 (h)	---
9:50-10:40	B2 (h)	C2 (g)	---	A2 (c)	D2 (a)
10:40-11:30	B2 (h)	A2 (c)	---	A2 (c)	D2 (a)
11:40-12:30	C2 (g)	A2 (c)	B2 (h)	C2 (g)	D2 (a)
12:30-13:20	C2 (g)	A2 (c)	B2 (h)	C2 (g)	---

 Grupo:  3
		lun	mar	mie	jue	vie
·····················································
8:00-8:50	B3 (f)	D3 (h)	B3 (f)	---	D3 (h)
8:50-9:40	B3 (f)	D3 (h)	B3 (f)	---	D3 (h)
9:50-10:40	C3 (g)	A3 (e)	B3 (f)	A3 (e)	D3 (h)
10:40-11:30	C3 (g)	A3 (e)	---	A3 (e)	C3 (g)
11:40-12:30	---	A3 (e)	C3 (g)	B3 (f

In [76]:
# imprimir variables por separado

for u_i in u_dict:
    u = u_dict[u_i]
    print(str(u) + " = ", u.variable.X)

for v_i in v_dict:
    v = v_dict[v_i]
    print(str(v) + " = ", v.variable.X)

for w_i in w_dict:
    w = w_dict[w_i]
    print(str(w) + " = ", w.variable.X)

# for r_i in r_dict:
#     r = r_dict[r_i]
#     print(str(r) + " = ", r.variable.X)

u_{A1_lun_8:00-8:50} =  0.0
u_{A1_lun_8:50-9:40} =  0.0
u_{A1_lun_9:50-10:40} =  -0.0
u_{A1_lun_10:40-11:30} =  -0.0
u_{A1_lun_11:40-12:30} =  0.0
u_{A1_lun_12:30-13:20} =  0.0
u_{A1_mar_8:00-8:50} =  0.0
u_{A1_mar_8:50-9:40} =  0.0
u_{A1_mar_9:50-10:40} =  1.0
u_{A1_mar_10:40-11:30} =  1.0
u_{A1_mar_11:40-12:30} =  1.0
u_{A1_mar_12:30-13:20} =  0.0
u_{A1_mie_8:00-8:50} =  -0.0
u_{A1_mie_8:50-9:40} =  -0.0
u_{A1_mie_9:50-10:40} =  0.0
u_{A1_mie_10:40-11:30} =  0.0
u_{A1_mie_11:40-12:30} =  0.0
u_{A1_mie_12:30-13:20} =  0.0
u_{A1_jue_8:00-8:50} =  0.0
u_{A1_jue_8:50-9:40} =  0.0
u_{A1_jue_9:50-10:40} =  1.0
u_{A1_jue_10:40-11:30} =  1.0
u_{A1_jue_11:40-12:30} =  -0.0
u_{A1_jue_12:30-13:20} =  0.0
u_{A1_vie_8:00-8:50} =  0.0
u_{A1_vie_8:50-9:40} =  0.0
u_{A1_vie_9:50-10:40} =  -0.0
u_{A1_vie_10:40-11:30} =  0.0
u_{A1_vie_11:40-12:30} =  -0.0
u_{A1_vie_12:30-13:20} =  -0.0
u_{B1_lun_8:00-8:50} =  1.0
u_{B1_lun_8:50-9:40} =  1.0
u_{B1_lun_9:50-10:40} =  -0.0
u_{B1_lun_10:40-11:30} =  -0.0


In [77]:
# imprimir calendario para profesores
print_prof_timetable(dias, horarios, u_dict, w_dict, profesores)


 Profesor:  a
		lun	mar	mie	jue	vie
·····················································
8:00-8:50	---	---	D2	---	---
8:50-9:40	---	---	D2	---	---
9:50-10:40	---	A1	---	A1	D2
10:40-11:30	---	A1	---	A1	D2
11:40-12:30	---	A1	---	---	D2
12:30-13:20	---	---	---	---	---

 Profesor:  b
		lun	mar	mie	jue	vie
·····················································
8:00-8:50	B1	---	---	---	---
8:50-9:40	B1	---	B1	---	---
9:50-10:40	---	---	B1	---	---
10:40-11:30	---	---	B1	---	---
11:40-12:30	---	---	---	---	B1
12:30-13:20	---	---	---	---	B1

 Profesor:  c
		lun	mar	mie	jue	vie
·····················································
8:00-8:50	---	C1	---	---	---
8:50-9:40	---	C1	---	---	---
9:50-10:40	C1	---	---	A2	---
10:40-11:30	C1	A2	---	A2	---
11:40-12:30	C1	A2	---	C1	---
12:30-13:20	---	A2	---	C1	---

 Profesor:  d
		lun	mar	mie	jue	vie
·····················································
8:00-8:50	---	---	---	---	D1
8:50-9:40	---	---	---	---	D1
9:50-10:40	---	---	---	---	D1
10:40-11:30	---	

In [78]:
model.write('timetable_opt.lp')

# model.display()